In [1]:
import os, time
import country_converter as coco

from osgeo import gdal
import pandas as pd 
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path 
import pycountry 

import rasterio as rs
from rasterio.plot import show
from rasterio.enums import Resampling
from rasterio.plot import plotting_extent
from rasterio.merge import merge
import rasterio.mask
from rasterio import shutil as rio_shutil
from rasterio.vrt import WarpedVRT

def tPrint(s):
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

In [2]:
path = Path('/Users/bjafino/Documents/GitHub/PovertyClimateFall2020/FATHOM')

# World Admin
worldAdmin_Folder = 'GMGD_world_admin'
admin_File = 'newold_geo_code2_povdata_v7.shp'

reproj_Folder = 'GMGD_reproj'
admin_File_proj = 'WB_CountryPolys.shp'

# Pluvial and fluvial flood files (Fathom) - 2019
flood_Folder = 'FLOOD_SSBN'
flood_subFolder_2019 = 'v2_2019'
fluvialFlood_Folder = 'fluvial_undefended'
fluvialFlood_FileName = 'FU_1in100.tif' 
pluvialFlood_Folder = 'pluvial'
pluvialFlood_FileName = 'P_1in100.tif' 

# Pluvial and fluvial flood files (Fathom) - 2016
flood_subFolder_2016 = 'v1_2016'
## The pluvial and fluvial folder names are prepended with each country's ISO2 code
## This is managed in the 2016 analysis loop

# Coastal flood files
coastalFlood_Folder = 'Coastal_flood'
coastalFlood_File = "ss_muis_rp0100m.tif"

# Population data
pop_Folder = 'Population'
pop_subFolder = 'WorldPop_PPP_2020'
pop_subsubFolder = 'MOSAIC_ppp_prj_2020'

# Folder paths
flood_path_2019 = path / flood_Folder / flood_subFolder_2019 
flood_path_2016 = path / flood_Folder / flood_subFolder_2016 
population_path = path / pop_Folder / pop_subFolder / pop_subsubFolder 

# Global files paths
adm_path_reproj = path / worldAdmin_Folder / reproj_Folder / admin_File_proj
adm_path = path / worldAdmin_Folder / admin_File
coastalFlood_path = path / coastalFlood_Folder / coastalFlood_File
 
print(flood_path_2019)
print(flood_path_2016)
print(population_path)
print(adm_path)
print(coastalFlood_path)

\Users\bjafino\Documents\GitHub\PovertyClimateFall2020\FATHOM\FLOOD_SSBN\v2_2019
\Users\bjafino\Documents\GitHub\PovertyClimateFall2020\FATHOM\FLOOD_SSBN\v1_2016
\Users\bjafino\Documents\GitHub\PovertyClimateFall2020\FATHOM\Population\WorldPop_PPP_2020\MOSAIC_ppp_prj_2020
\Users\bjafino\Documents\GitHub\PovertyClimateFall2020\FATHOM\GMGD_world_admin\newold_geo_code2_povdata_v7.shp
\Users\bjafino\Documents\GitHub\PovertyClimateFall2020\FATHOM\Coastal_flood\ss_muis_rp0100m.tif


In [3]:
outputPath = Path('/Users/bjafino/Documents/GitHub/PovertyClimateFall2020/FATHOM')

<a id='Previous'></a>
## Previously Processed Countries
If there are any previously processed countries, uncomment the second cell below. 
Otherwise, make sure the cell is commented out.

After commenting/uncommenting the previously processed:
- Run all cells until [here](#TOC), where you can then select the appropriate section


In [4]:
previouslyProcessed = []

In [5]:
# Comment out below if previously processed results do not exist or if you wish to rewrite

try:
    processed = gpd.read_file(outputPath /"Outputs"/"Results"/ "processed_countries.shp")
    previouslyProcessed = processed['ISO3'].unique()
    del processed
except:
    pass

In [6]:
previouslyProcessed

[]

## Creating folders for intermediate & final outputs

In [7]:
# Creating folder for all outputs
output_Folder = 'Outputs'
output_FolderPath = outputPath / output_Folder
try: 
    output_FolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [8]:
# Creating folder for raster outputs
Raster_outputFolder = 'FloodPop_Countries'
Raster_outputFolderPath = output_FolderPath / Raster_outputFolder
try: 
    Raster_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')
    

Already exists


In [9]:
# Creating folder for coastal flood country files
cFlood_outputFolder = 'CoastalFlood_Countries'
cFlood_outputFolderPath = output_FolderPath / cFlood_outputFolder
try: 
    cFlood_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [10]:
# Creating folder for aggregated flood country files (fluvial + pluvial + coastal)
Flood_outputFolder = 'Flood_Countries'
Flood_outputFolderPath = output_FolderPath / Flood_outputFolder
try: 
    Flood_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [11]:
# Creating folder for fluvial and pluvial merged flood files
PopAligned_outputFolder = 'Pop_aligned'
PopAligned_outputFolderPath = output_FolderPath / PopAligned_outputFolder
try: 
    PopAligned_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [12]:
# Creating folder for fluvial and pluvial merged flood files
FP_outputFolder = 'FluvialPluvial'
FP_outputFolderPath = output_FolderPath / FP_outputFolder
try: 
    FP_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [13]:
# Creating folder for final results
Results_outputFolder = 'Results'
Results_outputFolderPath = output_FolderPath / Results_outputFolder
try: 
    Results_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


## Flood Bins
0. No risk: x = 0
1. Limited risk: x <= 0.15
2. Moderate risk: 0.15 < x <= 0.5
3. High risk: 0.5 < x <= 1.5
4. Very high risk: x > 1.5
5. Water body: x >= 999


In [14]:
flood_bins = [1e-10,0.15,0.5,1.5,998,10000] 
# Right=True when digitizing to include right bin edge
# Note: 0 contains flood depths of 0 up to 0.0000000001 -> no flood
numberCategories = len(flood_bins)

## Result Dataframe

In [15]:
# An empty dataframe to store the results of each loop
result = pd.DataFrame(columns = ['OBJECTID',
                                '0-NoRiskPop',
                                '1-LowRiskPop', 
                                '2-ModerateRiskPop', 
                                '3-HighRiskPop', 
                                '4-VeryHighRiskPop', 
                                '5-WaterBodyPop',
                                'ISO3']).set_index('OBJECTID')

## Admin shapefiles

In [16]:
# Read shapefile
adm = gpd.read_file(adm_path)
# adm = gpd.read_file(adm_path_reproj)
adm = adm[['code','sample','region','geo_code2','geometry']]

In [17]:
adm.crs

{'init': 'epsg:4326'}

In [18]:
# Reproject to EPSG:4326
# adm = adm.to_crs('EPSG:4326')

In [19]:
# Correcting country codes
adm['ISO3'] = adm['code']
adm.ISO3 = adm.ISO3.replace('ADO', 'AND')
adm.ISO3 = adm.ISO3.replace('IMY', 'IMN')
adm.ISO3 = adm.ISO3.replace('URU', 'URY')
adm.ISO3 = adm.ISO3.replace('VNZ', 'VEN')
adm.ISO3 = adm.ISO3.replace('MOR', 'MAR')
#Kosovo WorldPop was originally KOS, then manually changed to XKX on the server

## Matching adm country names to folder/file names

In [20]:
# Get country codes from adm df
codeDF = adm[['ISO3']]
codeDF = codeDF.drop_duplicates(subset ="ISO3")
codeDF = codeDF.sort_values(by=['ISO3'])

# Put adm country names in list for easier comparison
admCodeList = codeDF.ISO3.values.tolist()
admList = coco.convert(names=admCodeList, to='name_short')
#admList.sort()

# Add country names to df
codeDF['ADM0_NAME'] = admList
adm = adm.merge(codeDF, on='ISO3', how='left')

C:\Users\bjafino\anaconda3\envs\GMTRA\lib\site-packages\country_converter\country_converter.py:412: FutureWarning: read_table is deprecated, use read_csv instead.
  for str_col in must_be_string})


In [21]:
# Get folder names for the Fathom data
list2019 = [os.path.basename(str(x)) for x in flood_path_2019.iterdir() if x.is_dir()]
# list2016 = [os.path.basename(str(x)) for x in flood_path_2016.iterdir() if x.is_dir()]

In [22]:
len(list2019)

226

In [23]:
# Match country names from lists - 2019
matching_dict_2019 = coco.match(admList, list2019);

## Correcting specific cases

In [24]:
# Sierra Leone has two folders, selecting one
if type(matching_dict_2019['Sierra Leone']) is list:
    matching_dict_2019['Sierra Leone'] = 'Sierra Leone'

In [25]:
# Fiji flood files are separated into Fiji_west and Fiji_east
for name in flood_path_2019.glob('Fiji_west'):
    fluvialFolder = flood_path_2019 / name / fluvialFlood_Folder
    FJfluvial = list(fluvialFolder.rglob('*100.tif'))
    
for name in flood_path_2019.glob('Fiji_east'):
    pluvialFolder = flood_path_2019 / name / pluvialFlood_Folder
    FJ_E_pluvial = list(pluvialFolder.rglob('*100.tif'))
    
for name in flood_path_2019.glob('Fiji_west'):
    pluvialFolder = flood_path_2019 / name / pluvialFlood_Folder
    FJ_W_pluvial = list(pluvialFolder.rglob('*100.tif'))

In [26]:
# Reorder so geometry is the last col
adm = adm[['ADM0_NAME','code','ISO3','sample','region','geo_code2','geometry']]

In [27]:
# from previous runs - combined 2016 and 2019 ,'PYF_1992'
bad_adm1 = ['FSM_2','TLS_201', 'BHR_2229','CAN_2253',
            'MAR_502','RUS_391','RUS_421','RUS_443','RUS_453',
            'RUS_2252','USA_759','HND_2117','MAR_502', 'CAN_558']

## Run for one country 2019

In [35]:
shp_name, flood_name = 'Afghanistan', 'afghanistan'

In [36]:
sub = adm.loc[adm['ADM0_NAME'] == shp_name].copy()

# Save country name
countryCode = sub['ISO3'].iloc[0]
countryNameInfo = pycountry.countries.get(alpha_3=countryCode)
tPrint(f'{countryCode}: {sub.shape[0]}') 

# Adm0 fluvial and pluvial processing
Fluvial_outputfile, Pluvial_outputfile = adm0_processing_2019(flood_name)

# # testing for one adm1 region
# sub = sub.iloc[[0]]
# Adm1 data processing loop:
result = adm1_loop(sub, result)

tPrint(f"{countryCode} - Completed")   

16:05:42	AFG: 34
16:05:45	AFG: Only one fluvial tile. No merge needed
16:05:51	AFG: Country level fluvial exported
16:05:55	AFG: Only one pluvial tile. No merge needed
16:06:02	AFG: Country level pluvial exported
16:06:03	AFG_1954 - Fluvial cropped to adm 1
16:06:05	AFG_1954 - Pluvial cropped to adm 1
16:06:09	AFG_1954 - popAligned aligned
16:06:10	AFG_1954 - Pop cropped to adm 1
16:06:11	AFG_1954 - Pop aligned & cropped. Total Pop: 1240509.0
16:06:12	AFG_1954 - AlignedCoastalFlood aligned
16:06:12	AFG_1954 - Coastal aligned
16:06:15	AFG_1954 - Flood mosaic merged & exported
16:06:16	AFG_1954 - FloodCropped cropped to adm 1
16:06:17	AFG_1954 - Confirming same shape. popArray shape: (1, 3661, 5886) floodArray shape: (1, 3661, 5886)
16:06:17	AFG_1954 - Flood categorized
16:06:18	AFG_1954 - Flood converted to bool int arrays
16:06:18	AFG_1954 - Flood multiplied by pop
16:06:24	AFG_1954 - Checksum: 1240504.875
16:06:24	AFG_1954 - Sums added to df


C:\Users\bjafino\anaconda3\envs\GMTRA\lib\site-packages\ipykernel\__main__.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


16:06:24	AFG_1955 - Fluvial cropped to adm 1
16:06:25	AFG_1955 - Pluvial cropped to adm 1
16:06:26	AFG_1955 - popAligned aligned
16:06:26	AFG_1955 - Pop cropped to adm 1
16:06:27	AFG_1955 - Pop aligned & cropped. Total Pop: 655911.375
16:06:27	AFG_1955 - AlignedCoastalFlood aligned
16:06:27	AFG_1955 - Coastal aligned
16:06:28	AFG_1955 - Flood mosaic merged & exported
16:06:28	AFG_1955 - FloodCropped cropped to adm 1
16:06:28	AFG_1955 - Confirming same shape. popArray shape: (1, 1819, 2867) floodArray shape: (1, 1819, 2867)
16:06:28	AFG_1955 - Flood categorized
16:06:28	AFG_1955 - Flood converted to bool int arrays
16:06:29	AFG_1955 - Flood multiplied by pop
16:06:30	AFG_1955 - Checksum: 655910.6875
16:06:30	AFG_1955 - Sums added to df
16:06:31	AFG_1956 - Fluvial cropped to adm 1
16:06:31	AFG_1956 - Pluvial cropped to adm 1
16:06:32	AFG_1956 - popAligned aligned
16:06:33	AFG_1956 - Pop cropped to adm 1
16:06:33	AFG_1956 - Pop aligned & cropped. Total Pop: 969238.625
16:06:33	AFG_1956 - 

16:08:11	AFG_1966 - Pluvial cropped to adm 1
16:08:11	AFG_1966 - popAligned aligned
16:08:12	AFG_1966 - Pop cropped to adm 1
16:08:12	AFG_1966 - Pop aligned & cropped. Total Pop: 4819151.5
16:08:12	AFG_1966 - AlignedCoastalFlood aligned
16:08:12	AFG_1966 - Coastal aligned
16:08:12	AFG_1966 - Flood mosaic merged & exported
16:08:12	AFG_1966 - FloodCropped cropped to adm 1
16:08:12	AFG_1966 - Confirming same shape. popArray shape: (1, 942, 1343) floodArray shape: (1, 942, 1343)
16:08:12	AFG_1966 - Flood categorized
16:08:12	AFG_1966 - Flood converted to bool int arrays
16:08:12	AFG_1966 - Flood multiplied by pop
16:08:13	AFG_1966 - Checksum: 4819153.5
16:08:13	AFG_1966 - Sums added to df
16:08:13	AFG_1967 - Fluvial cropped to adm 1
16:08:14	AFG_1967 - Pluvial cropped to adm 1
16:08:17	AFG_1967 - popAligned aligned
16:08:18	AFG_1967 - Pop cropped to adm 1
16:08:19	AFG_1967 - Pop aligned & cropped. Total Pop: 1419176.875
16:08:20	AFG_1967 - AlignedCoastalFlood aligned
16:08:20	AFG_1967 - C

16:09:02	AFG_1977 - Pluvial cropped to adm 1
16:09:03	AFG_1977 - popAligned aligned
16:09:04	AFG_1977 - Pop cropped to adm 1
16:09:04	AFG_1977 - Pop aligned & cropped. Total Pop: 471931.25
16:09:04	AFG_1977 - AlignedCoastalFlood aligned
16:09:04	AFG_1977 - Coastal aligned
16:09:05	AFG_1977 - Flood mosaic merged & exported
16:09:05	AFG_1977 - FloodCropped cropped to adm 1
16:09:06	AFG_1977 - Confirming same shape. popArray shape: (1, 2193, 2083) floodArray shape: (1, 2193, 2083)
16:09:06	AFG_1977 - Flood categorized
16:09:06	AFG_1977 - Flood converted to bool int arrays
16:09:06	AFG_1977 - Flood multiplied by pop
16:09:07	AFG_1977 - Checksum: 471931.0
16:09:07	AFG_1977 - Sums added to df
16:09:07	AFG_1978 - Fluvial cropped to adm 1
16:09:08	AFG_1978 - Pluvial cropped to adm 1
16:09:08	AFG_1978 - popAligned aligned
16:09:09	AFG_1978 - Pop cropped to adm 1
16:09:09	AFG_1978 - Pop aligned & cropped. Total Pop: 626945.1875
16:09:09	AFG_1978 - AlignedCoastalFlood aligned
16:09:09	AFG_1978 - 

## Run in loop

In [36]:
# 2019 bad countries - add ISO3 country code for any country that causes an issue
bad_countries_2019 = ['ABW','MDV','MHL','PYF','SYC','TUV','WSM'] #from Jun, old
bad_countries_2019 = ['ARG', 'ABW', 'ASM', 'BHR', 'BMU']
# from previous runs - combined 2016 and 2019 ,'PYF_1992'
bad_adm1 = ['CAN_2253']

In [37]:
try:
    result = pd.read_csv('Outputs/Results/temporary_results.csv', index_col=0)
except:
    pass

result.head()

,0-NoRiskPop,1-LowRiskPop,2-ModerateRiskPop,3-HighRiskPop,4-VeryHighRiskPop,5-WaterBodyPop,ISO3
1954,1051159.88,78725.62,45188.55,24185.02,40912.47,335.94,AFG
1955,570429.44,36018.29,23511.16,16800.77,9149.48,1.88,AFG
1956,682697.88,129411.83,76251.77,42780.76,38095.40,1.16,AFG
1957,953413.94,250442.92,67991.07,17319.62,19837.15,902.66,AFG
1958,455034.75,19323.88,14031.10,12753.71,10271.88,5.93,AFG


In [38]:
for shp_name, flood_name in matching_dict_2019.items():  
    if flood_name == "not_found":
#         None
        print(f"{shp_name} does not have flood data==================================")
    elif type(flood_name) == list and not flood_name == ['Fiji_east', 'Fiji_west']:
        print(f"{shp_name} has multiple datasets, need to investigate")
    else:
        # Skip processed countries
        processedCountries = result['ISO3'].unique()
        processedCountries = np.append(processedCountries, previouslyProcessed)

        sub = adm.loc[adm['ADM0_NAME'] == shp_name].copy()
        countryCode = sub['ISO3'].iloc[0]
        countryNameInfo = pycountry.countries.get(alpha_3=countryCode)
        
        if not countryCode in processedCountries and not countryCode in bad_countries_2019:
            tPrint(f'{countryCode}: {sub.shape[0]}') # start

            # Adm0 fluvial and pluvial processing
            Fluvial_outputfile, Pluvial_outputfile = adm0_processing_2019(flood_name)

            # Adm1 Loop:
            result = adm1_loop(sub, result)
            
            result.to_csv('Outputs/Results/temporary_results.csv')
                        
            tPrint(f"{countryCode} - Completed")  # end
        else:
            if countryCode in processedCountries:
                tPrint(f"{countryCode} already processed==================================")

15:05:06	AFG already processed==================================
15:05:06	AGO already processed==================================
15:05:06	ALB already processed==================================
15:05:06	AND already processed==================================
15:05:06	ARE already processed==================================
15:05:06	ARM already processed==================================
15:05:06	ATG already processed==================================
15:05:06	AUS already processed==================================
15:05:06	AUT already processed==================================
15:05:06	AZE already processed==================================
15:05:06	BDI already processed==================================
15:05:06	BEL already processed==================================
15:05:06	BEN already processed==================================
15:05:06	BFA already processed==================================
15:05:06	BGD already processed==================================
15:05:06	BGR already proc

C:\Users\bjafino\anaconda3\envs\GMTRA\lib\site-packages\ipykernel\__main__.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


15:24:02	CAN_548 - Sums added to df
15:24:03	Start processing CAN_549.
15:24:09	CAN_549 - Sums added to df
15:24:09	Start processing CAN_550.
15:25:03	CAN_550 - Sums added to df
15:25:03	Start processing CAN_551.
15:25:47	CAN_551 - Sums added to df
15:25:47	Start processing CAN_552.
15:37:40	CAN_552 - Sums added to df
15:37:42	Start processing CAN_553.
15:47:10	CAN_553 - Sums added to df
15:47:11	Start processing CAN_554.
15:52:03	CAN_554 - Sums added to df
15:52:04	Start processing CAN_555.
15:56:04	CAN_555 - Sums added to df
15:56:04	Start processing CAN_556.
16:00:14	CAN_556 - Sums added to df
16:00:14	Start processing CAN_557.
16:08:40	CAN_557 - Sums added to df
16:08:41	CAN_2253 is bad
16:08:41	Start processing CAN_2254.
16:22:30	CAN_2254 - adm1 memory error
16:22:35	Start processing CAN_2255.
16:27:45	CAN_2255 - Sums added to df
16:27:47	CAN - Completed
16:27:47	CHE: 7
16:27:47	CHE: Only one fluvial tile. No merge needed
16:27:47	CHE: Country level fluvial exported
16:27:48	CHE: 

18:23:10	COD_1570 - Sums added to df
18:23:10	Start processing COD_1571.
18:23:14	COD_1571 - Sums added to df
18:23:14	Start processing COD_1572.
18:23:52	COD_1572 - Sums added to df
18:23:52	Start processing COD_1573.
18:24:09	COD_1573 - Sums added to df
18:24:09	Start processing COD_1574.
18:26:03	COD_1574 - Sums added to df
18:26:03	Start processing COD_1575.
18:26:21	COD_1575 - Sums added to df
18:26:21	COD - Completed
18:26:21	COG: 12
18:26:23	COG: Only one fluvial tile. No merge needed
18:26:27	COG: Country level fluvial exported
18:26:30	COG: Only one pluvial tile. No merge needed
18:26:35	COG: Country level pluvial exported
18:26:35	Start processing COG_1553.
18:26:40	COG_1553 - Sums added to df
18:26:40	Start processing COG_1554.
18:26:40	COG_1554 - Sums added to df
18:26:40	Start processing COG_1555.
18:26:51	COG_1555 - Sums added to df
18:26:51	Start processing COG_1556.
18:26:59	COG_1556 - Sums added to df
18:26:59	Start processing COG_1557.
18:27:04	COG_1557 - Sums added t

18:39:51	DEU_249 - Sums added to df
18:39:51	Start processing DEU_250.
18:39:52	DEU_250 - Sums added to df
18:39:52	Start processing DEU_251.
18:40:05	DEU_251 - Sums added to df
18:40:05	Start processing DEU_252.
18:40:06	DEU_252 - Sums added to df
18:40:06	Start processing DEU_253.
18:40:08	DEU_253 - Sums added to df
18:40:08	Start processing DEU_254.
18:40:18	DEU_254 - Sums added to df
18:40:18	Start processing DEU_255.
18:40:28	DEU_255 - Sums added to df
18:40:28	Start processing DEU_256.
18:40:50	DEU_256 - Sums added to df
18:40:50	DEU - Completed
18:40:50	DJI: 6
18:40:50	DJI: Only one fluvial tile. No merge needed
18:40:50	DJI: Country level fluvial exported
18:40:51	DJI: Only one pluvial tile. No merge needed
18:40:51	DJI: Country level pluvial exported
18:40:51	Start processing DJI_1825.
18:40:53	DJI_1825 - Sums added to df
18:40:53	Start processing DJI_1826.
18:40:54	DJI_1826 - Sums added to df
18:40:54	Start processing DJI_1827.
18:40:56	DJI_1827 - Sums added to df
18:40:56	St

...

00:46:00	SVK: Only one pluvial tile. No merge needed
00:46:00	SVK: Country level pluvial exported
00:46:00	Start processing SVK_359.
00:46:02	SVK_359 - Sums added to df
00:46:02	Start processing SVK_360.
00:46:04	SVK_360 - Sums added to df
00:46:04	Start processing SVK_361.
00:46:06	SVK_361 - Sums added to df
00:46:06	Start processing SVK_362.
00:46:08	SVK_362 - Sums added to df
00:46:08	Start processing SVK_363.
00:46:11	SVK_363 - Sums added to df
00:46:11	Start processing SVK_364.
00:46:14	SVK_364 - Sums added to df
00:46:14	Start processing SVK_365.
00:46:17	SVK_365 - Sums added to df
00:46:17	Start processing SVK_366.
00:46:19	SVK_366 - Sums added to df
00:46:19	SVK - Completed
00:46:19	SVN: 1
00:46:20	SVN: Only one fluvial tile. No merge needed
00:46:20	SVN: Country level fluvial exported
00:46:20	SVN: Only one pluvial tile. No merge needed
00:46:20	SVN: Country level pluvial exported
00:46:20	Start processing SVN_950.
00:46:27	SVN_950 - Sums added to df
00:46:27	SVN - Completed
0

00:57:51	TGO_613 - Sums added to df
00:57:51	TGO - Completed
00:57:51	THA: 77
00:57:54	THA: Only one fluvial tile. No merge needed
00:58:01	THA: Country level fluvial exported
00:58:05	THA: Only one pluvial tile. No merge needed
00:58:12	THA: Country level pluvial exported
00:58:13	Start processing THA_120.
00:58:14	THA_120 - Sums added to df
00:58:14	Start processing THA_121.
00:58:14	THA_121 - Sums added to df
00:58:15	Start processing THA_122.
00:58:15	THA_122 - Sums added to df
00:58:15	Start processing THA_123.
00:58:16	THA_123 - Sums added to df
00:58:16	Start processing THA_124.
00:58:17	THA_124 - Sums added to df
00:58:17	Start processing THA_125.
00:58:18	THA_125 - Sums added to df
00:58:18	Start processing THA_126.
00:58:20	THA_126 - Sums added to df
00:58:20	Start processing THA_127.
00:58:21	THA_127 - Sums added to df
00:58:21	Start processing THA_128.
00:58:22	THA_128 - Sums added to df
00:58:22	Start processing THA_129.
00:58:23	THA_129 - Sums added to df
00:58:23	Start p

01:06:04	TUN_1856 - Sums added to df
01:06:04	Start processing TUN_1857.
01:06:04	TUN_1857 - Sums added to df
01:06:04	Start processing TUN_1858.
01:06:09	TUN_1858 - Sums added to df
01:06:09	Start processing TUN_1859.
01:06:14	TUN_1859 - Sums added to df
01:06:14	Start processing TUN_1860.
01:06:22	TUN_1860 - Sums added to df
01:06:22	Start processing TUN_1861.
01:06:39	TUN_1861 - Sums added to df
01:06:39	TUN - Completed
01:06:39	TUR: 1
01:06:42	TUR: Only one fluvial tile. No merge needed
01:06:49	TUR: Country level fluvial exported
01:06:53	TUR: Only one pluvial tile. No merge needed
01:07:01	TUR: Country level pluvial exported
01:07:01	Start processing TUR_367.
01:10:20	TUR_367 - Sums added to df
01:10:21	TUR - Completed
01:10:21	TUV: 1
01:10:21	TUV: No fluvial files found.
01:10:21	TUV: Only one pluvial tile. No merge needed
01:10:22	TUV: Country level pluvial exported
01:10:22	Start processing TUV_1134.
01:10:22	TUV_1134 - adm1 other error
01:10:22	TUV - Completed
Taiwan does not

01:22:32	UZB_374 - Sums added to df
01:22:32	Start processing UZB_375.
01:22:45	UZB_375 - Sums added to df
01:22:45	Start processing UZB_376.
01:22:52	UZB_376 - Sums added to df
01:22:52	Start processing UZB_377.
01:22:59	UZB_377 - Sums added to df
01:22:59	Start processing UZB_378.
01:23:32	UZB_378 - Sums added to df
01:23:32	Start processing UZB_379.
01:23:35	UZB_379 - Sums added to df
01:23:35	Start processing UZB_380.
01:23:41	UZB_380 - Sums added to df
01:23:41	Start processing UZB_381.
01:23:46	UZB_381 - Sums added to df
01:23:46	UZB - Completed
01:23:46	VCT: 6
01:23:46	VCT: No fluvial files found.
01:23:46	VCT: Only one pluvial tile. No merge needed
01:23:47	VCT: Country level pluvial exported
01:23:47	Start processing VCT_2096.
01:23:47	VCT_2096 - adm1 other error
01:23:47	Start processing VCT_2097.
01:23:47	VCT_2097 - adm1 other error
01:23:47	Start processing VCT_2098.
01:23:47	VCT_2098 - adm1 other error
01:23:47	Start processing VCT_2099.
01:23:47	VCT_2099 - adm1 other erro

01:51:12	ZWE: Country level pluvial exported
01:51:13	Start processing ZWE_614.
01:51:13	ZWE_614 - Sums added to df
01:51:13	Start processing ZWE_615.
01:51:14	ZWE_615 - Sums added to df
01:51:14	Start processing ZWE_616.
01:51:27	ZWE_616 - Sums added to df
01:51:27	Start processing ZWE_617.
01:51:35	ZWE_617 - Sums added to df
01:51:35	Start processing ZWE_618.
01:51:45	ZWE_618 - Sums added to df
01:51:45	Start processing ZWE_619.
01:52:00	ZWE_619 - Sums added to df
01:52:01	Start processing ZWE_620.
01:52:15	ZWE_620 - Sums added to df
01:52:15	Start processing ZWE_621.
01:52:35	ZWE_621 - Sums added to df
01:52:35	Start processing ZWE_622.
01:52:53	ZWE_622 - Sums added to df
01:52:53	Start processing ZWE_623.
01:53:11	ZWE_623 - Sums added to df
01:53:11	ZWE - Completed


# Functions

In [28]:
def get_FP_2019(flood_name, path_name, pfFlood_Folder):
  floodList = []
  for name in path_name.glob(flood_name):
      floodFolder = path_name / name / pfFlood_Folder
      if len(list(floodFolder.rglob('*100.tif'))) == 0:
          # Large countries have multiple FU_1in100-x.tif files (x = tile number)
          floodList = list(floodFolder.rglob('*100_tile_?.tif'))
      else:
          # Small countries have one FU_1in100.tif file
          floodList = list(floodFolder.rglob('*100.tif'))
  return floodList

def open_files_in_list(fileList):
    outputList = []
    for fp in fileList:
        src = rs.open(fp)
        outputList.append(src) 
    return outputList

def merge_export_adm0_FP_files(flood_files_list, outpath, countryCode, FP):
    if len(flood_files_list) >=1:
      # if multiple flood files exist, merge then export
      if len(flood_files_list) >1: 
          # Merge flood files.
          floodArray, out_trans = merge(flood_files_list, method='max')
          tPrint(f'{countryCode}: Country level {FP} merged') 
          ## Update the metadata
          out_meta = flood_files_list[0].meta.copy()
          out_meta.update({"height": floodArray.shape[1], 
                            "width": floodArray.shape[2],
                            "transform": out_trans})
      elif len(flood_files_list) ==1: 
          floodArray = flood_files_list[0].read()
          out_meta = flood_files_list[0].meta.copy()
          tPrint(f'{countryCode}: Only one {FP} tile. No merge needed')
      # export as a new geotiff 
      floodName = f"{FP}_{countryCode}.tif"
      merged_outputfile = outpath / floodName
      with rs.open(merged_outputfile, 'w', **out_meta, compress = 'LZW', tiled=True) as dest:
          dest.write(floodArray)
      tPrint(f'{countryCode}: Country level {FP} exported') 
    else:
      tPrint(f'{countryCode}: No {FP} files found.')
      merged_outputfile = outpath # fix or use this in next loop
    return merged_outputfile



In [29]:
def adm0_processing_2019(flood_name):
    # all other variables used are global
    ###################### Fluvial Adm0 ######################
    if flood_name == ['Fiji_east', 'Fiji_west']: 
      fluvialList = FJfluvial 
    else: 
      fluvialList = get_FP_2019(flood_name, flood_path_2019, fluvialFlood_Folder) 

    fluvialFiles = open_files_in_list(fluvialList)      
    Fluvial_outputfile = merge_export_adm0_FP_files(fluvialFiles, FP_outputFolderPath, countryCode, 'fluvial')

    ###################### Pluvial Adm0 ######################
    # Create list of pluvial flood files
    if flood_name == ['Fiji_east', 'Fiji_west']:
      pluvialList = FJ_E_pluvial + FJ_W_pluvial
    else:
      pluvialList = get_FP_2019(flood_name, flood_path_2019, pluvialFlood_Folder)

    pluvialFiles = open_files_in_list(pluvialList)  
    Pluvial_outputfile = merge_export_adm0_FP_files(pluvialFiles, FP_outputFolderPath, countryCode, 'pluvial')

    return Fluvial_outputfile, Pluvial_outputfile


In [30]:
def get_adm1_index(df_adm1):
    adm1Index = str(df_adm1.index)
    adm1Index = adm1Index.partition("[")[2].partition("]")[0]
    return adm1Index

def crop_to_shp(path_name, df, filePrefix):
    raster = rs.open(path_name)
    # Mask to shapefile
    out_array, out_trans = rs.mask.mask(dataset=raster, shapes=df.geometry, crop=True)

    out_meta = raster.meta.copy()
    out_meta.update({"height": out_array.shape[1], 
                      "width": out_array.shape[2],
                      "transform": out_trans})
    #tPrint(f"{countryCode}_{adm1Index} - {filePrefix} cropped to adm 1")
    return out_array, out_meta
    
def export_raster_to_geotiff(array, meta, path, fileName, filePrefix):
    Cropped_outputfile = path / fileName
    with rs.open(Cropped_outputfile, 'w', **meta, compress = 'LZW', tiled=True) as dest:
        dest.write(array)
    return Cropped_outputfile
    #tPrint(f"{countryCode}_{adm1Index} - {filePrefix} exported")
    
def vrtWarp(inpath, outpath, vrt_settings, country_code, adm_code, filePrefix):
    with rs.open(inpath) as src:
      with WarpedVRT(src, **vrt_settings) as vrt:
          data = vrt.read()
          # Process the dataset in chunks. 
          for _, window in vrt.block_windows(): data = vrt.read(window=window)
          # Export the aligned data 
          fileName = f"{filePrefix}_{country_code}_{adm_code}.tif"
          Aligned_outputfile = outpath / fileName        
          rio_shutil.copy(vrt, Aligned_outputfile, driver='GTiff')
    #tPrint(f"{countryCode}_{adm1Index} - {filePrefix} aligned")
    return Aligned_outputfile

def convert_to_bool_int_array(array):
      listNumberCat = range(numberCategories)
      list_IntBool_floodCat = []
      # Create a boolean int array for each category
      for i in listNumberCat:
          intArray = (array == i).astype(dtype = np.int8, copy = False)
          list_IntBool_floodCat.append(intArray)
      #tPrint(f"{countryCode}_{adm1Index} - Flood converted to bool int arrays")
      return list_IntBool_floodCat

def multiply_array_list(arrayList, array):
      flood_pop = []
      for i in range(len(arrayList)):
          flood_pop.append(np.multiply(arrayList[i], array))
      flood_pop = (np.vstack((flood_pop)))
      #tPrint(f"{countryCode}_{adm1Index} - Flood multiplied by pop")
      return flood_pop

# def multiply_array_list(arrayList, array):
# #     flood_pop = []
#     for i in range(len(arrayList)):
#         arrayList[i] = np.multiply(arrayList[i], array)
# #         flood_pop.append(np.multiply(arrayList[i], array))
# #         arrayList = arrayList[1:]
# #     flood_pop = (np.vstack((flood_pop)))
#     flood_pop = (np.vstack((arrayList)))
#     tPrint(f"{countryCode}_{adm1Index} - Flood multiplied by pop")
#     return flood_pop

def array3d_sum(array):
      Flood_pop_sums = {
        '0-NoRiskPop': np.sum(array[0]),
        '1-LowRiskPop': np.sum(array[1]),
        '2-ModerateRiskPop': np.sum(array[2]),
        '3-HighRiskPop': np.sum(array[3]),
        '4-VeryHighRiskPop': np.sum(array[4]),
        '5-WaterBodyPop': np.sum(array[5])
      }
      #tPrint(f"{countryCode}_{adm1Index} - Checksum: {np.sum(array)}")
      return Flood_pop_sums

def sums_to_df(sums, df):
      for key,value in sums.items() :
        df.loc[int(adm1Index), key] = value

      # rounding the values to x decimal places
      df.iloc[:,-6:] = df.iloc[:,-6:].apply(lambda x: round(x, 2));

      # Keeping only results cols to make merge easy
      df = df.iloc[:,-6:] 
      df['ISO3'] = countryCode
      return df

In [31]:
def adm1_processing(adm1, resultDF):
    ###################### Fluvial Adm1 ######################
    # crop
    floodArray, out_meta = crop_to_shp(Fluvial_outputfile, adm1, 'Fluvial')
    # export
    fluvialCroppedName = f"FU_{countryCode}_{adm1Index}.tif"
    FluvialCropped_outputfile = export_raster_to_geotiff(floodArray, out_meta, FP_outputFolderPath, 
                                                        fluvialCroppedName, 'Fluvial')
    
    # ###################### Pluvial Adm1 ######################
    # crop
    floodArray, out_meta = crop_to_shp(Pluvial_outputfile, adm1, 'Pluvial')
    # export
    pluvialCroppedName = f"PU_{countryCode}_{adm1Index}.tif"
    PluvialCropped_outputfile = export_raster_to_geotiff(floodArray, out_meta, FP_outputFolderPath, 
                                                        pluvialCroppedName, 'Pluvial')
    
    ###################### WarpedVRT ######################
    # Vritually warp the pop and coastal flood files to the fluvial and pluvial
    vrt_options ={
        'resampling': Resampling.nearest,
        'crs': out_meta['crs'],
        'transform' : out_meta['transform'],
        'height':floodArray.shape[1],
        'width': floodArray.shape[2]
    }

    ###################### Population ######################
    # Get population file
    pop_File = " "
    countryAlpha3 = f'*{countryCode}*'
    for name in population_path.glob(countryAlpha3):
        pop_File = name
    if pop_File != " ":
        # Align using Warped VRT
        popAligned_outputfile = vrtWarp(pop_File, PopAligned_outputFolderPath, vrt_options, countryCode, adm1Index, 'popAligned')
        pop = rs.open(popAligned_outputfile)
        # crop
        popArray, out_meta = crop_to_shp(popAligned_outputfile, adm1, 'Pop')
        # change dtype to reduce memory
        popArray = popArray.astype('float32', copy=False)
        out_meta.update({'dtype': 'float32'})
        # export
        popCropped = f"PopCropped_{countryCode}_{adm1Index}.tif"
        export_raster_to_geotiff(popArray, out_meta, PopAligned_outputFolderPath, popCropped, 'popCropped')
        # Remove negatives
        popArray[popArray < 0] = 0
        # Print population sum for country to compare against final results
        #tPrint(f"{countryCode}_{adm1Index} - Pop aligned & cropped. Total Pop: {np.sum(popArray)}")

        ###################### Coastal Adm1 ######################
        # Align using Warped VRT
        cFloodAligned_outputfile = vrtWarp(coastalFlood_path, cFlood_outputFolderPath,  
                                        vrt_options, countryCode, adm1Index, 'AlignedCoastalFlood')
        # At this point the coastal flood file is aligned and has been cropped to the pop bounding box
        # It will be cropped to the adm1 boundaries after merging with the fluvail and pluvial files
        #tPrint(f"{countryCode}_{adm1Index} - Coastal aligned")

        ###################### Flood mosaic adm1 ######################
        # Create flood mosaic
        combinedFloodList = [FluvialCropped_outputfile] + [PluvialCropped_outputfile] + [cFloodAligned_outputfile]            
        allFiles = []
        for fp in combinedFloodList:
            src = rs.open(fp)
            allFiles.append(src)

        # Merge flood files. Using max pixel by pixel method
        floodArray, out_trans = merge(allFiles, method='max')
        out_meta = allFiles[0].meta.copy()
        out_meta.update({"height": floodArray.shape[1], 
                          "width": floodArray.shape[2],
                          "transform": out_trans})
        
        # export
        floodNameMerged = f"FloodMerged_{countryCode}_{adm1Index}.tif"
        FloodMerged_outputfile = export_raster_to_geotiff(floodArray, out_meta, Flood_outputFolderPath, floodNameMerged, 'FloodMerged')
        #tPrint(f'{countryCode}_{adm1Index} - Flood mosaic merged & exported') 

        # ###################### Crop merged file flood ######################
        # crop
        floodArray, out_meta = crop_to_shp(FloodMerged_outputfile, adm1, 'FloodCropped')
        # export
        floodName = f"Flood_{countryCode}_{adm1Index}.tif"
        FloodCropped_outputfile = export_raster_to_geotiff(floodArray, out_meta, Flood_outputFolderPath, 
                                                            floodName, 'FloodCropped')
        #tPrint(f'{countryCode}_{adm1Index} - Confirming same shape. popArray shape: {popArray.shape} floodArray shape: {floodArray.shape}')

        ###################### Categorize by flood bins ######################
        floodArray = np.digitize(floodArray, flood_bins, right=True) # True to include right bin edge
        floodArray = floodArray.astype(dtype = np.int8, copy = False)
        #tPrint(f"{countryCode}_{adm1Index} - Flood categorized") 

        ###################### Convert to bool int arrays ######################
        list_IntBool_floodCat = convert_to_bool_int_array(floodArray)
        del floodArray

        ###################### Multiply flood array by pop array ######################
        # Multiply each int bool categorized flood array by the population array
        flood_pop = multiply_array_list(list_IntBool_floodCat, popArray)
        del list_IntBool_floodCat

        ##################### Export flood pop raster ######################
        # update meta
        out_meta.update({"nodata": 0., 'count': numberCategories})
        # export
        FloodPopName = f"FloodPop_{countryCode}_{adm1Index}.tif"
        export_raster_to_geotiff(flood_pop, out_meta, Raster_outputFolderPath, FloodPopName, 'flood pop stack')

        # ###################### Calculate sum for each exposure array ######################
        Flood_pop_sums = array3d_sum(flood_pop)

        #################################### Add to df ####################################
        adm1 = sums_to_df(Flood_pop_sums, adm1)
        resultDF = resultDF.append(adm1)
        tPrint(f"{countryCode}_{adm1Index} - Sums added to df")
    else:
        tPrint(f"{countryCode}_{adm1Index} - no pop file")
    return resultDF

In [32]:
def adm1_loop(sub_df, result_df):
  for namedTuple in sub_df.itertuples():
    adm1 = sub_df.loc[[int(namedTuple[0])]]
    adm1 = adm1[['ADM0_NAME','geometry']] 
    global adm1Index # making adm1Index global 
    adm1Index = get_adm1_index(adm1)
    adm1_name = f'{countryCode}_{adm1Index}' #countryCode is global
    # skip bad_adm1
    global bad_adm1 
    if adm1_name in bad_adm1: #bad_adm1 is global
        tPrint(f'{adm1_name} is bad')
    else:
        tPrint(f'Start processing {adm1_name}.')
        try: # main processing happens hhere
            result_df = adm1_processing(adm1, result_df) #result is global
        except rasterio.errors.WindowError:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 window error")
            continue
        except ValueError:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 value error")
            continue
        except MemoryError:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 memory error")
            continue
        except:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 other error")
            continue
  return result_df

## Merge & Exporting

In [39]:
# removing ISO3 col because it's in the results df
adm = adm[['ADM0_NAME', 'code', 'sample', 'region', 'geo_code2','geometry']]

In [40]:
# creating a merged df with only rows that have new results:
mergedDf1 = adm.merge(result, left_index=True, right_index=True)

In [41]:
# checking all looks good
mergedDf1.head()

,ADM0_NAME,code,sample,region,geo_code2,geometry,0-NoRiskPop,1-LowRiskPop,2-ModerateRiskPop,3-HighRiskPop,4-VeryHighRiskPop,5-WaterBodyPop,ISO3
0,"Micronesia, Fed. Sts.",FSM,1–Yap,EAP,FSM_2013_GADM1_FSM_FSM.4_1,"(POLYGON ((143.07833862 6.684165949999993, 143...",10205.07,467.60,190.62,62.79,65.78,0.97,FSM
1,"Micronesia, Fed. Sts.",FSM,2–Chuuk,EAP,FSM_2013_GADM1_FSM_FSM.1_1,"(POLYGON ((153.6361084 5.291110039999997, 153....",40862.93,1047.28,728.74,196.33,76.56,33.35,FSM
3,"Micronesia, Fed. Sts.",FSM,4–Kosrae,EAP,FSM_2013_GADM1_FSM_FSM.2_1,"(POLYGON ((162.96694946 5.281112189999991, 162...",3892.86,586.87,422.93,267.92,52.50,0.00,FSM
26,Australia,AUS,[1]New South Wales,EAP,AUS_2014_ADM1_470,"(POLYGON ((151.172748 -33.847877, 151.173537 -...",5999717.00,370404.81,227553.67,128814.43,50928.25,3555.13,AUS
27,Australia,AUS,[2]Victoria,EAP,AUS_2014_ADM1_476,"(POLYGON ((146.33117676 -39.14710998000001, 14...",4546959.00,398618.84,219808.27,119916.20,36683.57,282.05,AUS


In [42]:
# Exporting 
mergedDf1.to_file(Results_outputFolderPath / "processed_countries_1.shp")